In [8]:
def sanitize_for_obsidian(text: str) -> str:
    """Escape markdown so text is treated as plain in Obsidian."""
    lines = text.splitlines()
    escaped_lines = []
    special_chars = r'\`*_{}[]()#+-.!|>'
    for line in lines:
        escaped_line = ''.join(f'\\{c}' if c in special_chars else c for c in line)
        if escaped_line.lstrip().startswith('#'):
            # prevent heading by adding a zero-width space
            escaped_line = '\u200b' + escaped_line
        escaped_lines.append(escaped_line)
    return '\n'.join(escaped_lines)


In [9]:
from zero_liftsim import sandbox as sb
from codenamize import codenamize
from zero_helpers.imports import *
# sample time and agent, generate data. 
d = sb.get_sample()

fp = "/l/obs-chaotic/full context clear wipe prompt courageous-senior 467c6d36.md"
assert exists(fp)
with open(fp, 'r') as f:
    prompt = f.read()
with open('/l/m/m/sandbox.py', 'r') as f:
    sc = f.read()

prompt_data = {}
prompt_data['sc'] = sc
prompt_data['printme'] = d['printme']
printme = d['printme']

from jinja2 import Template

In [10]:
fp = "/l/obs-chaotic/prompt wrong-pleasure 908a7085.md"
with open(fp, 'r')  as f:
    template = Template(f.read())
prompt = template.render({'sc':sc, 'printme':d['printme'], 'keyParameters':d['keyParameters']})

i = d['keyParameters']['exp']['agent_uuid']
c = codenamize(i)

In [ ]:
from zero_consult_clouds import ChatGPT
#chat = ChatGPT()
#response = chat.send(prompt)

with open('/l/m/docs/unnamed-gruesome-church-737d4eba.md', 'a') as f:
    f.write(f'\n***\n# Example agent={c} {i}\n')
    f.write('\n***\n## response \n')
    f.write('xxx response here')
    f.write('\n***\n## Prompt \n')
    f.write('\n```prompt\n')
    f.write(sanitize_for_obsidian(prompt))
    f.write('\n```\n')
    f.write('\n***\n## Data \n')
    f.write('\n```data\n')
    f.write(sanitize_for_obsidian(printme))
    f.write('\n```\n')
    f.write('\n***\n')

with open('/l/obs-chaotic/prompt.md', 'w') as f:
    f.write(prompt)

In [46]:
sc = '{{ sc }}'
printme = '{{ printme }}'
prompt += f"\n***\n# simulation sample generation code\nthis code generates a sample of an agent and a time within a simulation. \n```python{sc}\n````\n is the output of that: \n***\n{printme}\n***\n\nnow, analyze values of `keyParams`. return a list of keys of keyParamaters that you think you can prove to be invalid. describe how each is provably invalid given what I've shared here. make sure to refer to the full signature of the value, e.g. keyParameters['a']['b''] or something ike that. next, as a separate task, generate a bullet point list for each value of keyParameters, but instead replace each with True (if the value can be provably possible)"

with open('/l/obs-chaotic/prompt.md', 'w') as f:
    f.write(prompt)

In [34]:
def collect_key_signatures_and_values(d, prefix="keyParameters"):
    result = []
    if isinstance(d, dict):
        for k, v in d.items():
            key_repr = f"{prefix}[{repr(k)}]"
            result.extend(collect_key_signatures_and_values(v, key_repr))
    else:
        result.append((prefix, d))
    return result

# Example usage:
signatures_and_values = collect_key_signatures_and_values(d['keyParameters'])

for k,v in dict(signatures_and_values).items():
    print(f"* <here you'll include True if you can prove this to be false, given the information given here. > {k} "+'= ({{'+f"{k}"+'}})'+'')

* <here you'll include True if you can prove this to be false, given the information given here. > keyParameters['cfg']['git_commit'] = ({{keyParameters['cfg']['git_commit']}})
* <here you'll include True if you can prove this to be false, given the information given here. > keyParameters['cfg']['SimulationManager']['__init__']['n_agents'] = ({{keyParameters['cfg']['SimulationManager']['__init__']['n_agents']}})
* <here you'll include True if you can prove this to be false, given the information given here. > keyParameters['cfg']['SimulationManager']['__init__']['lift_capacity'] = ({{keyParameters['cfg']['SimulationManager']['__init__']['lift_capacity']}})
* <here you'll include True if you can prove this to be false, given the information given here. > keyParameters['cfg']['SimulationManager']['__init__']['start_datetime'] = ({{keyParameters['cfg']['SimulationManager']['__init__']['start_datetime']}})
* <here you'll include True if you can prove this to be false, given the information

In [25]:
d['keyParameters']['cfg']['Lift']['num_chairs']

35

In [ ]:
from zero_helpers.imports import * 
from codenamize import codenamize as cd
from copy import deepcopy as copy
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 25, "lift_capacity": 4})
cfg['Lift']['num_chairs'] = 35

manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60*4)

# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

# sample an agent 
agent = manager.sample_agent()
INDEX_TIME = agent.get_activity_log_df()['time'].sample().iloc[0]

# this is the agent state @ time as self-determined (based on own logs, self.activity_log via self.get_latest_event)
state_gotten = agent.get_state(INDEX_TIME) 
event_latest = agent.get_latest_event(INDEX_TIME)

from zero_liftsim.agent_state_id import _EVENT_STATE_MAP as esm

edf = agent.get_rideloop_experience_log_df()
idx = edf.zero_liftsim.get_nearest_time_index(INDEX_TIME)
eid = edf.loc[idx]['exp_id']
print(f"* states are mapped to events like this: ")
for k,v in esm.items():
    print(f"    * {k} (event): {v} (action)")
print(f"""
* predicted agent state: {state_gotten}
* most recent event: {event_latest}
* experience (id): {cd(eid)} {eid}
* INDEX_TIME: {INDEX_TIME.strftime('%H:%M')}
""")
assert exp['exp_id'].is_unique
e_params = exp.set_index('exp_id').loc[eid].to_dict()

ldf = agent.get_activity_log_df()
l = agent.recent_agent_log_return_event_uuid(e_params['return_event_uuid'])

for k,v in e_params.items():
    print(f"* {k}: {v}")
    
print(f"Recent Activity Log")
print(l)
e_params

## analysis vars
keyParameters = {}
keyParameters['cfg'] = copy(cfg)
keyParameters['sample_time'] = INDEX_TIME
keyParameters['exp'] = e_params
keyParameters['state_map'] = esm
print(keyParameters)